In [494]:
import pandas as pd
import json
import copy

In [495]:
form_id = 1699353915355

In [496]:
file_path = f"../../../backend/source/forms/{form_id}.prod.json"

In [497]:
questions = []

In [498]:
with open(file_path, 'r') as json_file:
    data = json.load(json_file)
questions += [{
    "question_group": qg["id"],
    **q
} for qg in data["question_groups"] for q in qg["questions"]]

In [499]:
for q in questions:
    find_pre = next(
        (
            {
                item["name"]: {
                    item["pre"]["answer"]: list(
                        filter(lambda f: f["id"] == q["id"], item["pre"]["fill"])
                    )[0]["answer"]
                }
            }
            for item in questions if any(
                fill_item["id"] == q["id"] and "pre" not in q
                for fill_item in item.get("pre", {}).get("fill", [])
            )
        ),
        None
    )
    if find_pre:
        q["pre"] = find_pre

In [500]:
for q in questions:
    if "pre" in q and "fill" in q["pre"]:
        del q["pre"]

In [501]:
df = pd.json_normalize(data)

In [502]:
df2 = pd.DataFrame(df["question_groups"][0])

In [503]:
def group_mapping(row):
    qs = list(filter(lambda q: q["question_group"] == row["id"], questions))
    qs = copy.deepcopy(qs)
    for q in qs:
        del q["question_group"]
    row["questions"] = qs
    return row
df2 = df2.apply(group_mapping, axis=1)

In [504]:
df["question_groups"] = df["question_groups"].apply(lambda x: df2.to_dict("records"))

In [505]:
j = df.to_dict('records')[0]

In [506]:
with open(f"../../../backend/source/forms/{form_id}.prod.json", 'w') as f:
    json.dump(j, f)